In [28]:
import configparser
import os, argparse

In [40]:
import rasterio
import numpy as np
from scipy.ndimage import distance_transform_edt
import geopandas as gpd

In [50]:
# get user inputs from config file:
config_file = os.path.abspath("config.ini")
config = configparser.ConfigParser()
config.read(config_file)
config_dict = config['inputs']

def _parse_args(args):
    parser = argparse.ArgumentParser(
        description="",
        epilog="""
            """,
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    # parser.add_argument("-rd", "--remove_duplicates", help="Removing duplicate geometries after merging?, default to True", default=True)
    
    return parser.parse_args(args)


In [51]:
##################################
# Provide your inputs here
##################################
LOG_FILE = config_dict['log_file']
road = config_dict['road']

In [53]:
### store all the data into dataframe

roads = gpd.read_file(road)

In [ ]:
##################################
# Provide a log to trace the process
##################################
import logging

logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logging.getLogger().addHandler(logging.StreamHandler())
_str_decorator = "=" * 20
logging.info(f"\n{_str_decorator} BEGINNING LOG {_str_decorator}")

In [31]:
def compute_deforestation(
    forest_t1_path,
    forest_t2_path,
    output_path,
    nodata=None
):
    """
    Compute deforestation: pixels that were forest in t1 but not forest in t2

    Args:
        forest_t1_path (str): Path to t1 forest raster
        forest_t2_path (str): Path to t2 forest raster
        output_path (str): Output raster path
        nodata (int, float, optional): nodata value to mask (default: None)
    """
    with rasterio.open(forest_t1_path) as src1, rasterio.open(forest_t2_path) as src2:
        t1 = src1.read(1)
        t2 = src2.read(1)
        profile = src1.profile.copy()

        if nodata is not None:
            mask = (t1 == nodata) | (t2 == nodata)
        else:
            mask = np.zeros_like(t1, dtype=bool)

        # Deforestation: forest in t1 (1), non-forest in t2 (0)
        change = np.where((t1 == 1) & (t2 == 0) & ~mask, 1, 0).astype("uint8")
        if nodata is not None:
            change[mask] = nodata
            profile["nodata"] = nodata

        profile.update(dtype="uint8", count=1)
        with rasterio.open(output_path, "w", **profile) as dst:
            dst.write(change, 1)

    print(f"Deforestation raster saved to {output_path}")
    return output_path
    

In [35]:
def euclidean_distance(
    input_raster,
    output_raster,
    dist_meters=100 # distance unit to output
):

    """
    Calculates Euclidean distance to the nearest raster pixel and outputs distance in units of dist_meters
    
    Args:
        input_raster (str): Path to geotiff
        output_raster (str): Output raster path
        dist_meters (float): Distance unit (e.g., 100 for 100m units)
    """

    with rasterio.open(input_raster) as src:
        mask = src.read(1)
        profile = src.profile.copy()
        pixel_size = src.transform[0]

    invert_mask = (mask == 0).astype(np.uint8)
    dist_pixels = distance_transform_edt(invert_mask)
    dist_in_units = (dist_pixels * pixel_size) / dist_meters

    profile.update(dtype='float32')
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(dist_in_units.astype("float32"), 1)

    print(f"Euclidean distance raster saved as {output_raster}")
    return output_path

In [33]:
compute_deforestation(forest_cover_t1, forest_cover_t2, "input/forest_cover_change_t12.tif", nodata=None)
compute_deforestation(forest_cover_t2, forest_cover_t3, "input/forest_cover_change_t23.tif", nodata=None)

Deforestation raster saved to data/forest_cover_change_t23.tif


'data/forest_cover_change_t23.tif'

In [38]:
euclidean_distance("input/forest_cover_change_t12.tif", "input/distance_forest_change_t12.tif", 100)
euclidean_distance("input/forest_cover_change_t23.tif", "input/distance_forest_change_t23.tif", 100)

Eculidean distance raster saved as input/distance_forest_change_t12.tif


'intermediary/distance_from_forest_t1.tif'

In [59]:
import Rasterize

road_rasterizer = Rasterize(shp_path=road, target_crs=54304)

road_rasterizer.rasterize("input/road.tif", pixel_size=30)

IndentationError: unindent does not match any outer indentation level (Rasterize.py, line 13)